In [1]:
from IPython.display import display, HTML, IFrame

In [2]:
from bootlets.templates import *

In [3]:
def print_html(html):
    display(HTML(html))

`bootlets.templates` contains classes for most (one day all) HTML tags (e.g. `Div()` for `<div>`
When by default when instanciated, `*args` are joined by a newline, and `**kwargs` are used as keywords for the HTML tag. For example `P('This is come content', class_='bold').draw()` returns `<p class="bold">This is some content</p>` 

In [4]:
P('This is come content', class_='bold').draw()

'<p class="bold">This is come content</p>'

The HTML objects can be nested within one another, like you would with the HTML:

In [5]:
html = Container(
            H('Header 1', size=1),
            P('paragraph'),
            Div(P('seperate set of'), P('paragraphs', class_='bold')),
            UlList(*['List', 'of', 'things']),
            DlDict({'title': 'description', 
                             'another title': 'another description',
                             'nested list': UlList(*['one', 'two'])
                            })
        )

In [6]:
print(html.draw())

<h1>Header 1</h1>
<p>paragraph</p>
<div><p>seperate set of</p>
<p class="bold">paragraphs</p></div>
<ul><li>List</li>
<li>of</li>
<li>things</li></ul>
<dl class="row"><dt class="col-sm-3">title</dt><dd class="col-sm-9">description</dd>
<dt class="col-sm-3">another title</dt><dd class="col-sm-9">another description</dd>
<dt class="col-sm-3">nested list</dt><dd class="col-sm-9"><ul><li>one</li>
<li>two</li></ul></dd></dl>


In [7]:
print_html(html.draw())

The HTML object `__repr__()` will return the entire object construction

In [8]:
print(html)

Container(H('Header 1', size=1), P('paragraph'), Div(P('seperate set of'), P('paragraphs', class_='bold')), UlList('List', 'of', 'things'), DlDict({'title': 'description', 'another title': 'another description', 'nested list': UlList('one', 'two')}))


In [9]:
exec_str = html.__repr__()
exec('exec_rtn = '+exec_str)
print(exec_rtn.draw())

<h1>Header 1</h1>
<p>paragraph</p>
<div><p>seperate set of</p>
<p class="bold">paragraphs</p></div>
<ul><li>List</li>
<li>of</li>
<li>things</li></ul>
<dl class="row"><dt class="col-sm-3">title</dt><dd class="col-sm-9">description</dd>
<dt class="col-sm-3">another title</dt><dd class="col-sm-9">another description</dd>
<dt class="col-sm-3">nested list</dt><dd class="col-sm-9"><ul><li>one</li>
<li>two</li></ul></dd></dl>


The classes can be subclasses, and defaults overwritten:

In [10]:
class FancyDiv(Div):
    defaults = {'class_': 'styled'}

In [11]:
fd = FancyDiv(H('Header'), P('some content'))

In [12]:
fd.draw()

'<div class="styled"><h1>Header</h1>\n<p>some content</p></div>'

Alternatively, you can instanciate the class then call the object return a new instance overwriting the content or kwargs.

In [19]:
p = P('original content', class_='styleA')

In [20]:
p.draw()

'<p class="styleA">original content</p>'

In [21]:
p('new content').draw()

'<p class="styleA">new content</p>'

In [22]:
p('more content', class_='styleB').draw()

'<p class="styleB">more content</p>'

In [23]:
p(class_='styleC').draw()

'<p class="styleC">original content</p>'